In [1]:
import numpy as np
import pandas as pd
import os
import re
from glob import glob

In [11]:
G32_raw_data = sorted(glob('./raw_data/G32/G*.txt'))

In [12]:
G32_raw_data_name = list(map(lambda x:x[15:],G32_raw_data))

In [13]:
i = 0
all_data = pd.DataFrame()
for file in G32_raw_data:
    data = pd.read_csv(file, sep = '\t')
    data['data_name'] = G32_raw_data_name[i]
    data.columns = data.columns.str.replace('  ','')
    data.columns = data.columns.str.replace('G32 -','G32-') #有些奇怪的間隔
    data['data_point'] = data.index
    all_data = pd.concat([data,all_data])
    all_data = all_data.drop([0]) #消除單位Deg.
    all_data.reset_index(drop = True)
    i = i + 1

C:\Users\AutoUser\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [14]:
all_data.columns

Index(['G32-PTC1', 'G32-PTC10', 'G32-PTC11', 'G32-PTC12', 'G32-PTC13',
       'G32-PTC14', 'G32-PTC15', 'G32-PTC16', 'G32-PTC17', 'G32-PTC18',
       'G32-PTC19', 'G32-PTC2', 'G32-PTC20', 'G32-PTC21', 'G32-PTC22',
       'G32-PTC23', 'G32-PTC24', 'G32-PTC25', 'G32-PTC26', 'G32-PTC27',
       'G32-PTC28', 'G32-PTC29', 'G32-PTC3', 'G32-PTC30', 'G32-PTC31',
       'G32-PTC32', 'G32-PTC33', 'G32-PTC34', 'G32-PTC35', 'G32-PTC36',
       'G32-PTC37', 'G32-PTC38', 'G32-PTC39', 'G32-PTC4', 'G32-PTC40',
       'G32-PTC41', 'G32-PTC42', 'G32-PTC43', 'G32-PTC44', 'G32-PTC45',
       'G32-PTC47', 'G32-PTC5', 'G32-PTC6', 'G32-PTC7', 'G32-PTC8', 'G32-PTC9',
       'Unnamed: 8', 'data_name', 'data_point'],
      dtype='object')

In [15]:
#drop 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 8' 的columns
#rows出現 Deg的資料 欄位
for column in all_data.columns:
    if column[0] == 'U':
        all_data.drop([column],axis = 1,inplace = True)

In [16]:
all_data['data_name'] = all_data['data_name'].apply(lambda x:x[:-11]) #去掉 _Export.txt

In [17]:
all_data['data_name'].tail()

345    G32-AC(12X30)20160411-002
346    G32-AC(12X30)20160411-002
347    G32-AC(12X30)20160411-002
348    G32-AC(12X30)20160411-002
349    G32-AC(12X30)20160411-002
Name: data_name, dtype: object

In [18]:
all_data['recipes'] = all_data['data_name'].str.extract(r'(G\d+)',expand = False) #抓出所屬的Recipes
all_data['autoclave'] = all_data['data_name'].str.extract(r'(AC\(\d+x\d+\))',expand = False,flags=re.IGNORECASE) #用正則抓出 所屬的熱壓爐
all_data['date'] = all_data['data_name'].str.extract(r'(\d{7,8})',expand = False) #抓出日期
all_data['number'] = all_data['data_name'].str.extract(r'(00\d{1,3})',expand = False)#抓出所屬編號
all_data.drop(['data_name'],axis = 1,inplace = True)

In [19]:
all_data.columns = all_data.columns.str.replace('G32-','')

In [20]:
PTC_list = []
for column in all_data.columns:
    if column[0] == 'P' and column[-1] != 't':
        PTC_list.append(column)

In [21]:
all_data['PTC_count'] = len(PTC_list) - (all_data.shape[1] - all_data.count(axis = 1)) 
#每筆資料所收集溫度參數的個數

In [22]:
all_data.columns

Index(['PTC1', 'PTC10', 'PTC11', 'PTC12', 'PTC13', 'PTC14', 'PTC15', 'PTC16',
       'PTC17', 'PTC18', 'PTC19', 'PTC2', 'PTC20', 'PTC21', 'PTC22', 'PTC23',
       'PTC24', 'PTC25', 'PTC26', 'PTC27', 'PTC28', 'PTC29', 'PTC3', 'PTC30',
       'PTC31', 'PTC32', 'PTC33', 'PTC34', 'PTC35', 'PTC36', 'PTC37', 'PTC38',
       'PTC39', 'PTC4', 'PTC40', 'PTC41', 'PTC42', 'PTC43', 'PTC44', 'PTC45',
       'PTC47', 'PTC5', 'PTC6', 'PTC7', 'PTC8', 'PTC9', 'data_point',
       'recipes', 'autoclave', 'date', 'number', 'PTC_count'],
      dtype='object')

In [23]:
all_data['TemParameter'] = 0

In [24]:
G32_data = pd.DataFrame()

In [25]:
for PTC in PTC_list:
    data = all_data.loc[~all_data[PTC].isnull(),
             ['number', 'date', 'autoclave', 'recipes'
              ,'TemParameter', 'PTC_count','data_point',PTC]] #將各別 PTC 非0的數據挑出
    data['TemParameter'] = PTC  #建立 TemParameter 對應所屬來的PTC
    data['value'] = data[PTC]  # 賦予value值 統一
    data.drop([PTC],inplace=True,axis = 1)
    G32_data = pd.concat([G32_data,data])

In [26]:
G32_data.reset_index(drop=True, inplace=True)

In [27]:
G32_data['TemParameter'].value_counts()

PTC33    4224
PTC11    3589
PTC9     3519
PTC31    3225
PTC26    3198
PTC17    3177
PTC2     3096
PTC8     3075
PTC14    2794
PTC29    2789
PTC32    2535
PTC23    2510
PTC25    2480
PTC36    2469
PTC37    2441
PTC4     2421
PTC42    2188
PTC30    2138
PTC6     2134
PTC13    2112
PTC22    2102
PTC3     2083
PTC12    2080
PTC34    2075
PTC41    1813
PTC7     1770
PTC1     1770
PTC15    1760
PTC19    1757
PTC28    1712
PTC5     1454
PTC27    1435
PTC39    1412
PTC38    1401
PTC18    1400
PTC16    1367
PTC45    1042
PTC44    1040
PTC21     742
PTC10     724
PTC20     722
PTC40     719
PTC43     714
PTC47     393
PTC24     340
PTC35     339
Name: TemParameter, dtype: int64

In [28]:
G32_data.to_csv('./training_data/G32_data.csv')